In [1]:
import sys
import torch
import wandb
sys.path.append('..')
import pytorch_lightning as pl
from utils.data_modules.super_resolution import EEGSuperResolutionDataModule
from models.trainers.super_resolution import SuperResolutionTrainerModel, SaveSuperResEpochsCallbackMinimal

s:\PolySecLabProjects\eeg-image-decoding\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


S:\PolySecLabProjects\eeg-image-decoding\data\all-joined-1\eeg\epochs


In [2]:
dm = EEGSuperResolutionDataModule(
    input_channels=['Fp1', 'Fp2', 'AF3', 'AF4', 'F7', 'F3', 'Fz', 'F4', 'F8', 'FT7', 'FC3', 'FCz', 'FC4', 'FT8', 'T7', 'C3', 'Cz', 'C4', 'T8', 'TP7', 'CP3', 'CPz', 'CP4', 'TP8', 'P7', 'P3', 'Pz', 'P4', 'P8', 'O1', 'Oz', 'O2'],
    output_channels=['Fp1', 'AF7', 'AF3', 'F1', 'F3', 'F5', 'F7', 'FT7', 'FC5', 'FC3', 'FC1', 'C1', 'C3', 'C5', 'T7', 'TP7', 'CP5', 'CP3', 'CP1', 'P1', 'P3', 'P5', 'P7', 'P9', 'PO7', 'PO3', 'O1', 'Iz', 'Oz', 'POz', 'Pz', 'CPz', 'Fpz', 'Fp2', 'AF8', 'AF4', 'AFz', 'Fz', 'F2', 'F4', 'F6', 'F8', 'FT8', 'FC6', 'FC4', 'FC2', 'FCz', 'Cz', 'C2', 'C4', 'C6', 'T8', 'TP8', 'CP6', 'CP4', 'CP2', 'P2', 'P4', 'P6', 'P8', 'P10', 'PO8', 'PO4', 'O2'],
    sfreq=250,
    montage='standard_1020',
    window_before_event_ms=50,
    window_after_event_ms=600,
    subject=1, 
    session=1, 
    batch_size=64, 
    num_workers=4,
    test='All'
)

In [3]:
sample_data = dm.get_sample_info()
epochs = 100
subject = 1
session = 1
input_channels = sample_data['input']['channel_names']
output_channels = sample_data['output']['channel_names']
timesteps = sample_data['input']['num_timesteps']

Using ALL data for subject=1, session=1 as TEST SET
Found 3839 total samples for test set
Data split - Train: 2879, Val: 320, Test: 3839
Creating Datasets...
Original dataframe size: 2879
Original dataframe size: 320
Original dataframe size: 3839


In [4]:
checkpoint_path = "S:\\PolySecLabProjects\\eeg-image-decoding\\code\\models\\check_points\\super_resolution\\subj1_session1_epoch=99.ckpt"

In [5]:
lightning_model = SuperResolutionTrainerModel.load_from_checkpoint(checkpoint_path)

save_callback = SaveSuperResEpochsCallbackMinimal(
    save_dir="S:\\PolySecLabProjects\\eeg-image-decoding\\data\\all-joined-1\\eeg\\epochs\\650ms-250Hz", 
    filename="subj{subject}_session{session}_epochs_sr"
)

trainer = pl.Trainer(
    max_epochs=epochs,
    callbacks=[save_callback],
    accelerator='gpu' if torch.cuda.is_available() else 'cpu',
    devices=1
)

trainer.test(lightning_model, dm)

s:\PolySecLabProjects\eeg-image-decoding\env\Lib\site-packages\pytorch_lightning\utilities\migration\utils.py:56: The loaded checkpoint was produced with Lightning v2.5.2, which is newer than your current Lightning version: v2.5.1.post0
Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
s:\PolySecLabProjects\eeg-image-decoding\env\Lib\site-packages\pytorch_lightning\trainer\connectors\logger_connector\logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]

Using ALL data for subject=1, session=1 as TEST SET
Found 3839 total samples for test set
Data split - Train: 2879, Val: 320, Test: 3839
Creating Datasets...
Original dataframe size: 2879
Original dataframe size: 320


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Original dataframe size: 3839
Testing DataLoader 0: 100%|██████████| 60/60 [07:38<00:00,  0.13it/s]
=== NMSE ANALYSIS ===
NMSE (variance): 0.527355
NMSE (mean square): 0.527331
Average MSE: 0.256146
Average Target Variance: 0.485718
Average Target Mean Square: 0.485741


=== SUPER RESOLUTION TEST RESULTS ===
Test Loss: 17.708557
Test SNR: 2.7417 dB
Test MAE: 0.357683
Test MSE: 0.256158
Test NMSE: 0.527355
Test Pearson Correlation: 0.7655

Saved 3839 super-resolution epochs to: S:\PolySecLabProjects\eeg-image-decoding\data\all-joined-1\eeg\epochs\650ms-250Hz\subj{subject}_session{session}_epochs_sr.npy
Testing DataLoader 0: 100%|██████████| 60/60 [07:38<00:00,  0.13it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss            17.708

[{'test_loss': 17.70855712890625,
  'test_snr': 2.7416791915893555,
  'test_mae': 0.3576827943325043,
  'test_mse': 0.2561577260494232,
  'test_nmse': 0.5273549556732178,
  'test_pearson': 0.7655397653579712}]